In [1]:
%%writefile app.py

"""
Sentiment Analysis Dashboard (Streamlit)
Fast baseline (TF-IDF + LogisticRegression
CSV upload & batch prediction
Manual text input for single prediction
Visualizations (pie, bar, confusion matrix)

"""

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.pipeline import Pipeline
import joblib
import plotly.express as px

# Text cleaning
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Caching and warnings
from functools import partial
import warnings
warnings.filterwarnings("ignore")

# Initialize NLTK resources
nltk_downloader_done = False
try:
    stopwords.words("english")
    nltk_downloader_done = True
except Exception:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk_downloader_done = True

EN_STOPS = set(stopwords.words("english"))
STEMMER = SnowballStemmer("english")

#Helper functions

def clean_text(text: str) -> str:
    """Basic cleaning: lowercase, remove URLs, punctuation, stopwords, and stem."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove urls
    text = re.sub(r"[^a-zA-Z\s]", " ", text)  # letters only
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in EN_STOPS and len(t) > 1]
    stems = [STEMMER.stem(t) for t in tokens]
    return " ".join(stems)

@st.cache_data(show_spinner=False)
def preprocess_series(series: pd.Series) -> pd.Series:
    return series.fillna("").astype(str).apply(clean_text)

# Fast baseline training pipeline (TF-IDF + LR)
@st.cache_resource
def train_baseline(X_texts, y_labels):
    """Train and return a pipeline and vectorizer+model for quick inference."""
    tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
    clf = LogisticRegression(max_iter=200, C=1.0, solver='liblinear')
    pipeline = Pipeline([
        ("tfidf", tfidf),
        ("clf", clf)
    ])
    pipeline.fit(X_texts, y_labels)
    return pipeline

# Predict helper
def predict_df(pipeline, texts: pd.Series):
    preds = pipeline.predict(texts)
    probs = pipeline.predict_proba(texts) if hasattr(pipeline, "predict_proba") else None
    return preds, probs

# Utility: metrics summary
def get_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average='weighted', zero_division=0),
        "recall": recall_score(y_true, y_pred, average='weighted', zero_division=0),
        "f1": f1_score(y_true, y_pred, average='weighted', zero_division=0)
    }

# Save & load baseline model (joblib)
MODEL_PATH = "baseline_sentiment_model.joblib"
def save_model(pipeline, path=MODEL_PATH):
    joblib.dump(pipeline, path)

def load_model(path=MODEL_PATH):
    if os.path.exists(path):
        return joblib.load(path)
    return None

# Streamlit UI

st.set_page_config(page_title="Sentiment Analysis Dashboard", layout="wide", initial_sidebar_state="expanded")
st.title("📝 Sentiment Analysis Dashboard")
st.markdown(
    """
    Upload a CSV containing text and optionally labels (e.g. 'text' and 'label' columns).
    Quick baseline model (TFIDF + Logistic Regression) is trained instantly.
    Toggle the *Advanced* panel to fine-tune DistilBERT (requires GPU).
    """
)

# Sidebar controls
st.sidebar.header("Configuration")
text_column = st.sidebar.text_input("Text column name (CSV)", value="text")
label_column = st.sidebar.text_input("Label column name (CSV, optional)", value="label (optional)")
train_fraction = st.sidebar.slider("Train fraction (if labels present)", 0.1, 0.9, 0.8)

# Fine-tune options
st.sidebar.markdown("---")
use_distilbert = st.sidebar.checkbox("Enable DistilBERT fine-tuning (advanced)", value=False)
if use_distilbert:
    st.sidebar.info("DistilBERT requires 'transformers', 'datasets' and ideally a GPU. Use cautiously.")
    bert_epochs = st.sidebar.number_input("Epochs", min_value=1, max_value=10, value=2)
    bert_batch = st.sidebar.selectbox("Batch size", [8, 16, 32], index=1)
    bert_lr = st.sidebar.number_input("Learning rate", 1e-6, 1e-3, value=2e-5, format="%.6f")

# Upload & sample
st.subheader("1) Upload data (CSV) or use sample")
uploaded_file = st.file_uploader("Upload CSV file (must include text column)", type=["csv"])
sample_button, sample_col = st.columns([1,3])
with sample_col:
    st.markdown("Or use a small sample dataset (IMDB subset) for demo:")

use_sample = st.button("Load Demo Sample (small IMDB)")

if use_sample:
    # small demo dataset created inline (so there's no external dependency)
    demo_texts = [
        "I loved this movie, it was fantastic and thrilling!",
        "Horrible experience. The product broke on day one.",
        "It was okay, not the best but not the worst.",
        "Excellent service and very friendly staff.",
        "Terrible. Will never buy again."
    ]
    demo_labels = ["positive", "negative", "neutral", "positive", "negative"]
    df = pd.DataFrame({"text": demo_texts, "label": demo_labels})
else:
    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
        except Exception as e:
            st.error(f"Failed to read CSV: {e}")
            st.stop()
    else:
        # empty state
        st.info("Upload a CSV or press 'Load Demo Sample' to try the app.")
        df = pd.DataFrame()

if not df.empty:
    st.write("Dataset preview:")
    st.dataframe(df.head(10))

    # Validate text column
    if text_column not in df.columns:
        # try to auto-detect a text-like column
        text_candidates = [c for c in df.columns if df[c].dtype == object or df[c].dtype == 'string']
        if text_candidates:
            detected = st.selectbox("Detected text columns - choose one", options=text_candidates)
            text_column = detected
            st.sidebar.text_input("Text column name (CSV)", value=text_column)
        else:
            st.error(f"Could not find a text column named '{text_column}'. Please upload valid CSV.")
            st.stop()

    texts_raw = df[text_column].astype(str).fillna("")

    # If label column exists and is in df
    y = None
    has_labels = False
    if label_column in df.columns:
        y = df[label_column].astype(str).fillna("")
        has_labels = True
    else:
        # If user provided a label name but not present, warn; else proceed unlabeled.
        if label_column.strip() and label_column != "label (optional)":
            st.warning(f"Label column '{label_column}' not found — running in unlabeled mode.")
        has_labels = False

    # Preprocess texts
    with st.spinner("Cleaning text (cached)..."):
        texts_clean = preprocess_series(texts_raw)

    # If labels present, prepare and show distribution
    if has_labels:
        st.subheader("Label distribution")
        label_counts = y.value_counts().rename_axis('label').reset_index(name='counts')
        fig = px.pie(label_counts, names='label', values='counts', title='Label distribution')
        st.plotly_chart(fig, use_container_width=True)

    # If labeled we can train/test baseline
    if has_labels and st.button("Train baseline model (TF-IDF + LogisticRegression)"):
        # split
        X_train, X_test, y_train, y_test = train_test_split(
            texts_clean, y, train_size=train_fraction, random_state=42, stratify=y if has_labels else None
        )
        with st.spinner("Training baseline (cached) ..."):
            baseline_pipeline = train_baseline(X_train, y_train)
            # Save model
            save_model(baseline_pipeline)
        st.success("Baseline trained and cached!")

        # Evaluate
        y_pred = baseline_pipeline.predict(X_test)
        metrics = get_metrics(y_test, y_pred)
        st.subheader("Baseline performance (on test set)")
        st.write(metrics)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        fig2, ax2 = plt.subplots(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test), ax=ax2)
        ax2.set_xlabel("Predicted")
        ax2.set_ylabel("Actual")
        ax2.set_title("Confusion Matrix")
        st.pyplot(fig2)

        st.subheader("Classification report")
        st.text(classification_report(y_test, y_pred, zero_division=0))

    # If user wants to load existing baseline model
    if st.button("Load saved baseline model (if exists)"):
        loaded = load_model()
        if loaded is not None:
            st.success("Loaded saved baseline model.")
            baseline_pipeline = loaded
        else:
            st.warning("No saved baseline model found. Train one first.")
            baseline_pipeline = None

    # Single-text prediction UI
    st.subheader("Single Text Prediction")
    input_text = st.text_area("Type text here for live prediction", height=120)
    predict_button = st.button("Predict this text")
    if predict_button:
        if not input_text.strip():
            st.warning("Please enter text to predict.")
        else:
            # prefer using loaded/trained pipeline if available, else quick train on all data if labels present
            pipeline = load_model()
            if pipeline is None:
                # if labels are present, train quickly on whole dataset; else train on demo (no labels -> can't)
                if has_labels:
                    with st.spinner("Training quick baseline on all labeled data..."):
                        pipeline = train_baseline(texts_clean, y)
                        save_model(pipeline)
                else:
                    st.error("No trained model available and no labels to train on.")
                    pipeline = None
            if pipeline is not None:
                cleaned = clean_text(input_text)
                pred = pipeline.predict([cleaned])[0]
                proba = pipeline.predict_proba([cleaned])[0] if hasattr(pipeline, "predict_proba") else None
                st.write(f"**Predicted label:** {pred}")
                if proba is not None:
                    probs_df = pd.DataFrame([proba], columns=pipeline.classes_)
                    st.write("**Probabilities:**")
                    st.dataframe(probs_df.T.rename(columns={0:"probability"}))

    # Batch prediction & download
    st.subheader("Batch prediction & download")
    if st.button("Run batch prediction on uploaded data"):
        pipeline = load_model()
        if pipeline is None:
            st.warning("No saved baseline model found. Training baseline quickly on all labeled data (if labels exist)...")
            if has_labels:
                with st.spinner("Training baseline on all labeled data..."):
                    pipeline = train_baseline(texts_clean, y)
                    save_model(pipeline)
            else:
                st.error("No labels to train on — please upload labeled CSV or train manually.")
                pipeline = None

        if pipeline is not None:
            with st.spinner("Predicting..."):
                preds, probs = predict_df(pipeline, texts_clean)
                df_out = df.copy()
                df_out["predicted_label"] = preds
                if probs is not None:
                    # Add probability columns
                    prob_df = pd.DataFrame(probs, columns=[f"prob_{c}" for c in pipeline.classes_])
                    df_out = pd.concat([df_out.reset_index(drop=True), prob_df.reset_index(drop=True)], axis=1)
                csv = df_out.to_csv(index=False).encode('utf-8')
                st.download_button("Download predictions CSV", data=csv, file_name="predictions.csv", mime="text/csv")
                st.success("Predictions ready and downloadable.")

    # Advanced: DistilBERT fine-tuning
    if use_distilbert:
        st.subheader("DistilBERT Fine-tuning (Advanced)")
        st.markdown("""
            **Notes:**
            Fine-tuning requires 'transformers', 'datasets', and ideally a GPU.
            This action can take a long time on CPU and may fail without enough memory.
            Use only on medium-sized labeled datasets (thousands of rows) and when comfortable.
        """)
        if not has_labels:
            st.warning("DistilBERT fine-tuning requires labeled data. Upload a labeled CSV and try again.")
        else:
            do_finetune = st.button("Start DistilBERT fine-tuning (this will run here!)")
            if do_finetune:
                try:
                    from datasets import Dataset
                    from transformers import (
                        AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
                    )
                    # Prepare a small HF dataset
                    st.info("Preparing dataset for transformers...")
                    # Map labels to numeric
                    unique_labels = y.unique().tolist()
                    label2id = {lab: i for i, lab in enumerate(unique_labels)}
                    y_numeric = y.map(label2id)
                    ds = Dataset.from_pandas(pd.DataFrame({"text": texts_raw, "label": y_numeric}))
                    ds = ds.train_test_split(test_size=0.2, seed=42)

                    model_name = "distilbert-base-uncased"
                    tokenizer = AutoTokenizer.from_pretrained(model_name)
                    def tokenize_fn(batch):
                        return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
                    ds = ds.map(tokenize_fn, batched=True)
                    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

                    # Model
                    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_labels))

                    training_args = TrainingArguments(
                        output_dir="./distilbert_sentiment",
                        evaluation_strategy="epoch",
                        save_strategy="epoch",
                        learning_rate=bert_lr,
                        per_device_train_batch_size=bert_batch,
                        per_device_eval_batch_size=bert_batch,
                        num_train_epochs=bert_epochs,
                        weight_decay=0.01,
                        logging_dir="./logs",
                        load_best_model_at_end=True,
                    )

                    def compute_metrics_eval(eval_pred):
                        logits, labels = eval_pred
                        preds = np.argmax(logits, axis=-1)
                        return {
                            "accuracy": (preds == labels).mean()
                        }

                    trainer = Trainer(
                        model=model,
                        args=training_args,
                        train_dataset=ds['train'],
                        eval_dataset=ds['test'],
                        compute_metrics=compute_metrics_eval
                    )
                    st.info("Training started (this may take a long time). Outputs will be saved to ./distilbert_sentiment")
                    trainer.train()
                    st.success("Fine-tuning finished. Model saved to ./distilbert_sentiment")
                except Exception as e:
                    st.error(f"DistilBERT training failed or libraries missing: {e}")
                    st.info("Ensure `transformers`, `datasets`, and `torch` are installed and you have sufficient RAM/GPU.")
st.markdown("---")
st.caption("Built with ❤️  contact me if you'd like a custom version for your business Regard Taimour_AI.")

Writing app.py


In [4]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 142.9 MB/s eta 0:00:00


In [6]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.99.62:8501

  Stopping...
  Stopping...
^C
